weaves

Rough working: pyvote and Bell Ordinal number calculation

In [1]:
import os
import datetime

import pandas as pd
import numpy as np
import functools
import scipy.special as scis
from functools import partial
from itertools import permutations, combinations, chain
import string

import weaves 

%load_ext autoreload
%autoreload 2

In [2]:
from py3votecore.plurality import PluralityAtLarge
from py3votecore.stv import STV, Quota
from py3votecore.schulze_by_graph import SchulzeMethodByGraph, SchulzeNPRByGraph

import pyvote

In [3]:
print(pyvote.__version__)
print(pyvote.__Id__)

0.1.0
$Id: e6c1cce8af7f88d8635a28819453f91ae0cb6f85 $


In [4]:
# Generate data
input = {
    ('a', 'b'): 4,
    ('b', 'a'): 3,
    ('a', 'c'): 4,
    ('c', 'a'): 3,
    ('b', 'c'): 4,
    ('c', 'b'): 3,
}
judge0 = SchulzeMethodByGraph(input)
output = judge0.as_dict()
output

{'candidates': {'a', 'b', 'c'},
 'winner': 'a',
 'pairs': {('b', 'a'): 3,
  ('b', 'c'): 4,
  ('c', 'a'): 3,
  ('c', 'b'): 3,
  ('a', 'b'): 4,
  ('a', 'c'): 4},
 'strong_pairs': {('b', 'c'): 4, ('a', 'b'): 4, ('a', 'c'): 4}}

    3 voters preferring candidate A to B to C
    1 voter preferring candidate B to C to A
    1 voter preferring candidate C to A to B
    1 voter preferring candidate C to B to A

input = {
    ('a', ('b', 'c')): 3,
    ('b', 'c', 'a'): 1,
    ('c', 'a', 'b'): 1,
    ('c', 'b', 'a'): 1,
}
judge0 = SchulzeMethodByGraph(input)
output = judge0.as_dict()
output

In [5]:
judge0.graph
judge0.candidates

{'a', 'b', 'c'}

In [6]:
def graph_winner(self):
    losing_candidates = set([edge[1] for edge in self.graph.edges()])
    winning_candidates = set(self.graph.nodes()) - losing_candidates
    if len(winning_candidates) == 1:
        self.winner = list(winning_candidates)[0]
    elif len(winning_candidates) > 1:
        self.tied_winners = winning_candidates
        self.winner = self.break_ties(winning_candidates)
    else:
        self.condorcet_completion_method()
            
    return self

In [7]:
graph_winner(judge0)

In [8]:
judge0.winner

'a'

In [9]:
type(judge0)

py3votecore.schulze_by_graph.SchulzeMethodByGraph

In [10]:
judge0.graph.nodes()

['b', 'c', 'a']

In [11]:
judge0.graph.edges()

[('b', 'c'), ('a', 'b'), ('a', 'c')]

The permutohedron of order n has n! vertices, each of which is adjacent to n − 1 others, so the total number of edges is (n − 1)n!/2. Each edge has length √2, and connects two vertices that differ by swapping two coordinates the values of which differ by one.[3]

In [12]:
l = [0, 1, 2]
l1 = permutations(l, len(l))
l1 = map(np.array, l1)
l1 = list(l1)
print(len(l1), l1)

6 [array([0, 1, 2]), array([0, 2, 1]), array([1, 0, 2]), array([1, 2, 0]), array([2, 0, 1]), array([2, 1, 0])]


In [13]:
l2=list(combinations(l1, len(l)-1))
print(len(l2), l2)
abs(l2[0][0] - l2[0][1])

15 [(array([0, 1, 2]), array([0, 2, 1])), (array([0, 1, 2]), array([1, 0, 2])), (array([0, 1, 2]), array([1, 2, 0])), (array([0, 1, 2]), array([2, 0, 1])), (array([0, 1, 2]), array([2, 1, 0])), (array([0, 2, 1]), array([1, 0, 2])), (array([0, 2, 1]), array([1, 2, 0])), (array([0, 2, 1]), array([2, 0, 1])), (array([0, 2, 1]), array([2, 1, 0])), (array([1, 0, 2]), array([1, 2, 0])), (array([1, 0, 2]), array([2, 0, 1])), (array([1, 0, 2]), array([2, 1, 0])), (array([1, 2, 0]), array([2, 0, 1])), (array([1, 2, 0]), array([2, 1, 0])), (array([2, 0, 1]), array([2, 1, 0]))]


array([0, 1, 1])

Try building a permutohedron

In [14]:
weaves.POSetOps.instance().strong('ABC')

{('A',),
 ('A', 'B'),
 ('A', 'B', 'C'),
 ('A', 'C'),
 ('A', 'C', 'B'),
 ('B',),
 ('B', 'A'),
 ('B', 'A', 'C'),
 ('B', 'C'),
 ('B', 'C', 'A'),
 ('C',),
 ('C', 'A'),
 ('C', 'A', 'B'),
 ('C', 'B'),
 ('C', 'B', 'A')}

In [15]:
weaves.POSetOps.instance().adjacency(tuple('ABC'))

{'m': {0: 'A', 1: 'B', 2: 'C'},
 'n': ((0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)),
 'e': (((0, 1, 2), (0, 2, 1)),
  ((0, 1, 2), (1, 0, 2)),
  ((0, 2, 1), (1, 2, 0)),
  ((1, 0, 2), (2, 0, 1)),
  ((1, 2, 0), (2, 1, 0)),
  ((2, 0, 1), (2, 1, 0)))}

In [16]:
weaves.POSetOps.instance().adjacency(tuple('ABCD'))

{'m': {0: 'A', 1: 'B', 2: 'C', 3: 'D'},
 'n': ((0, 1, 2, 3),
  (0, 1, 3, 2),
  (0, 2, 1, 3),
  (0, 2, 3, 1),
  (0, 3, 1, 2),
  (0, 3, 2, 1),
  (1, 0, 2, 3),
  (1, 0, 3, 2),
  (1, 2, 0, 3),
  (1, 2, 3, 0),
  (1, 3, 0, 2),
  (1, 3, 2, 0),
  (2, 0, 1, 3),
  (2, 0, 3, 1),
  (2, 1, 0, 3),
  (2, 1, 3, 0),
  (2, 3, 0, 1),
  (2, 3, 1, 0),
  (3, 0, 1, 2),
  (3, 0, 2, 1),
  (3, 1, 0, 2),
  (3, 1, 2, 0),
  (3, 2, 0, 1),
  (3, 2, 1, 0)),
 'e': (((0, 1, 2, 3), (0, 1, 3, 2)),
  ((0, 1, 2, 3), (0, 2, 1, 3)),
  ((0, 1, 2, 3), (1, 0, 2, 3)),
  ((0, 1, 3, 2), (0, 2, 3, 1)),
  ((0, 1, 3, 2), (1, 0, 3, 2)),
  ((0, 2, 1, 3), (0, 3, 1, 2)),
  ((0, 2, 1, 3), (1, 2, 0, 3)),
  ((0, 2, 3, 1), (0, 3, 2, 1)),
  ((0, 2, 3, 1), (1, 2, 3, 0)),
  ((0, 3, 1, 2), (0, 3, 2, 1)),
  ((0, 3, 1, 2), (1, 3, 0, 2)),
  ((0, 3, 2, 1), (1, 3, 2, 0)),
  ((1, 0, 2, 3), (1, 0, 3, 2)),
  ((1, 0, 2, 3), (2, 0, 1, 3)),
  ((1, 0, 3, 2), (2, 0, 3, 1)),
  ((1, 2, 0, 3), (1, 3, 0, 2)),
  ((1, 2, 0, 3), (2, 1, 0, 3)),
  ((1, 2, 3, 0), (1, 

In [17]:
def build(candidates, ballots):
    graph = digraph()
    graph.add_nodes(candidates)
    for pair in itertools.permutations(candidates, 2):
        graph.add_edge(pair, sum([
        ballot["count"]
        for ballot in ballots
        if ballot["ballot"][pair[0]] > ballot["ballot"][pair[1]]
    ]))
    return graph

In [18]:
from itertools import permutations, combinations
import numpy as np

from functools import partial

In [19]:
import weaves

d0 = weaves.POSetOps.instance().sym2basis('ABC')

c0 = weaves.POSetOps.instance().adjacency(d0)
c0

{'m': {0: 'A', 1: 'B', 2: 'C'},
 'n': ((0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)),
 'e': (((0, 1, 2), (0, 2, 1)),
  ((0, 1, 2), (1, 0, 2)),
  ((0, 2, 1), (1, 2, 0)),
  ((1, 0, 2), (2, 0, 1)),
  ((1, 2, 0), (2, 1, 0)),
  ((2, 0, 1), (2, 1, 0)))}

In [21]:
d0.keys()

dict_keys([0, 1, 2])

In [ ]:
def remap(xs, d1=None):
    return [ d1[x] for x in xs ]

def join0(x, s=None):
    return x

def join1(x, s="-"):
    return s.join(x)

f = partial(join1, s="-")

remap1 = partial(remap, d1=d0)

for x in c0['n']:
    print(x, remap1(x), f(remap1(x)))

In [57]:
c0['e']

(((0, 1, 2), (0, 2, 1)),
 ((0, 1, 2), (1, 0, 2)),
 ((0, 2, 1), (1, 2, 0)),
 ((1, 0, 2), (2, 0, 1)),
 ((1, 2, 0), (2, 1, 0)),
 ((2, 0, 1), (2, 1, 0)))

In [58]:
[ ( f(remap1(x[0])), f(remap1(x[1])) ) for x in c0['e'] ]

[('A-B-C', 'A-C-B'),
 ('A-B-C', 'B-A-C'),
 ('A-C-B', 'B-C-A'),
 ('B-A-C', 'C-A-B'),
 ('B-C-A', 'C-B-A'),
 ('C-A-B', 'C-B-A')]

In [32]:
z0 = zip( [ c0['n'], [ remap1(x) for x in c0['n']] ])


ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [59]:
g0

NameError: name 'g0' is not defined